In [43]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

from utils import *

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, SplineTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge, Lars, SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

from statsmodels.regression.linear_model import RegressionResults

import warnings
warnings.filterwarnings("ignore")

In [44]:
dataframe = pd.read_csv("house_price.csv")
OUTPUT_COLUMN = 'Цена'
numeric = ['Количество.комнат', 'Общая.площадь', 'Этаж', 'Этажей.в.доме','Школа.1000', 'ВУЗ.1000',
       'ТЦ.1000', 'Стоматология.1000', 'Почта.1000', 'Поликлиника.1000',
       'Парк.1000', 'Остановка.1000', 'Одежда..1000', 'Супер3кет.1000',
       'Кинотеатр.1000', 'Кафе.1000', 'АЗС.1000', 'Детский.сад.1000',
       'Бар.1000', 'Банк.1000', 'Аптека.1000']
dataframe.head(5)

,id,Количество.комнат,Студия,Общая.площадь,Этаж,Этажей.в.доме,Парковка,Без.ремонта,Дизайнерский,Евроремонт,Косметический,Балкон,Лоджия,Совмещенный.санузел,Раздельный.санузел,Лифт,Грузовой.лифт,Мусоропровод,Год.постройки,Аварийный,Железобетонные.перекрытия,Смешанные.перекрытия,Деревянные.перекрытия,Иные.перекрытия,Панельные.стены,Блочные.стены,Деревянные.стены,Кирпичные.стены,Монолитные.стены,Смешанные.стены,Школа.1000,ВУЗ.1000,ТЦ.1000,Стоматология.1000,Почта.1000,Поликлиника.1000,Парк.1000,Остановка.1000,Одежда..1000,Супер3кет.1000,Кинотеатр.1000,Кафе.1000,АЗС.1000,Детский.сад.1000,Бар.1000,Банк.1000,Аптека.1000,Цена
0,896,2,0,46.0,3,5,0,0,0,0,1,1,0,1,0,0,0,0,1963,0,1,0,0,0,1,0,0,0,0,0,8,5,2,7,3,1,1,11,2,3,0,7,3,13,2,7,18,2350000.0
1,399,2,0,44.6,5,5,0,0,0,0,1,1,0,1,0,0,0,0,1969,0,1,0,0,0,1,0,0,0,0,0,3,1,1,5,2,2,1,8,3,5,0,7,3,11,1,4,11,1799000.0
2,1515,2,0,53.0,1,9,0,1,0,0,0,0,0,0,0,1,0,1,1977,0,1,0,0,0,1,0,0,0,0,0,6,1,5,8,4,6,1,13,4,11,0,5,3,17,3,7,15,2800000.0
3,1216,2,0,64.0,4,5,0,0,0,1,0,1,0,1,0,0,0,0,1966,0,1,0,0,0,0,0,0,1,0,0,5,25,8,15,3,7,1,16,31,5,1,55,2,10,28,32,17,3970000.0
4,677,1,0,33.3,1,10,0,0,1,0,0,0,0,1,0,0,2,1,2006,0,1,0,0,0,0,0,0,0,1,0,1,0,0,2,1,1,0,2,1,3,0,2,3,9,0,3,6,1880000.0


# Предобработка

In [45]:
# Пропусков в данных нет
dataframe.isna().isna().sum()

id                           0
Количество.комнат            0
Студия                       0
Общая.площадь                0
Этаж                         0
Этажей.в.доме                0
Парковка                     0
Без.ремонта                  0
Дизайнерский                 0
Евроремонт                   0
Косметический                0
Балкон                       0
Лоджия                       0
Совмещенный.санузел          0
Раздельный.санузел           0
Лифт                         0
Грузовой.лифт                0
Мусоропровод                 0
Год.постройки                0
Аварийный                    0
Железобетонные.перекрытия    0
Смешанные.перекрытия         0
Деревянные.перекрытия        0
Иные.перекрытия              0
Панельные.стены              0
Блочные.стены                0
Деревянные.стены             0
Кирпичные.стены              0
Монолитные.стены             0
Смешанные.стены              0
Школа.1000                   0
ВУЗ.1000                     0
ТЦ.1000 

In [46]:
# Категориальные признаки уже закодированы, однако всё еще необходимо удалить по одному столбцу для каждого закодированного признака
feature_groups = [['Без.ремонта', 'Дизайнерский', 'Евроремонт', 'Косметический'],
                  ['Балкон', 'Лоджия'], 
                  ['Железобетонные.перекрытия', 'Смешанные.перекрытия','Деревянные.перекрытия', 'Иные.перекрытия'],
                  ['Панельные.стены', 'Блочные.стены', 'Деревянные.стены', 'Кирпичные.стены', 'Монолитные.стены', 'Смешанные.стены']]

dataframe.drop(columns = [group[-1] for group in feature_groups], inplace=True)

feature_groups = [group[:-1] for group in feature_groups]
# Сохраняем оставшиеся категориальные данные
categorical = []
[categorical.extend(group)for group in feature_groups]



[None, None, None, None]

In [47]:
categorical

['Без.ремонта',
 'Дизайнерский',
 'Евроремонт',
 'Балкон',
 'Железобетонные.перекрытия',
 'Смешанные.перекрытия',
 'Деревянные.перекрытия',
 'Панельные.стены',
 'Блочные.стены',
 'Деревянные.стены',
 'Кирпичные.стены',
 'Монолитные.стены']

In [48]:
# Обработка аномальных значений
print(f"Размер исходной таблицы - {len(dataframe)}")
for column in numeric:
    dataframe = sigma_clip(dataframe, column)
print(f"Размер новой таблицы - {len(dataframe)}")

Размер исходной таблицы - 808
Размер новой таблицы - 691


In [49]:
# Удаление признаков, слабо коррелирующих с целевой переменной. Вероятно ухудшает результат
dataframe = remove_weakly_correlated(dataframe, OUTPUT_COLUMN)
# Удаление сильно коррелирующих друг с другом признаков
dataframe = remove_highly_correlated(dataframe, OUTPUT_COLUMN)

numeric = list(set(numeric).intersection(dataframe.columns))

dataframe.head(100)

,Студия,Общая.площадь,Этаж,Парковка,Евроремонт,Балкон,Совмещенный.санузел,Раздельный.санузел,Мусоропровод,Год.постройки,Блочные.стены,ВУЗ.1000,Остановка.1000,Кафе.1000,Бар.1000,Цена
0,0,46.0,3,0,0,1,1,0,0,1963,0,5,11,7,2,2350000.0
1,0,44.6,5,0,0,1,1,0,0,1969,0,1,8,7,1,1799000.0
2,0,53.0,1,0,0,0,0,0,1,1977,0,1,13,5,3,2800000.0
3,0,33.3,1,0,0,0,1,0,1,2006,0,0,2,2,0,1880000.0
4,0,45.0,8,0,0,0,1,0,1,2007,0,0,8,8,4,2870000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,79.0,8,0,0,0,0,1,0,1969,0,2,12,15,2,4000000.0
96,0,64.4,5,0,0,0,0,1,0,1975,0,3,12,10,2,3800000.0
97,0,32.0,4,0,0,1,0,1,0,1976,0,0,9,6,1,1350000.0
98,1,28.0,1,0,0,0,0,0,1,1976,0,7,9,10,6,1750000.0


In [50]:
dataframe[numeric] = MinMaxScaler().fit_transform(dataframe[numeric])
dataframe.head(5)

,Студия,Общая.площадь,Этаж,Парковка,Евроремонт,Балкон,Совмещенный.санузел,Раздельный.санузел,Мусоропровод,Год.постройки,Блочные.стены,ВУЗ.1000,Остановка.1000,Кафе.1000,Бар.1000,Цена
0,0,0.327273,0.117647,0,0,1,1,0,0,1963,0,0.416667,0.407407,0.368421,0.142857,2350000.0
1,0,0.314545,0.235294,0,0,1,1,0,0,1969,0,0.083333,0.296296,0.368421,0.071429,1799000.0
2,0,0.390909,0.000000,0,0,0,0,0,1,1977,0,0.083333,0.481481,0.263158,0.214286,2800000.0
3,0,0.211818,0.000000,0,0,0,1,0,1,2006,0,0.000000,0.074074,0.105263,0.000000,1880000.0
4,0,0.318182,0.411765,0,0,0,1,0,1,2007,0,0.000000,0.296296,0.421053,0.285714,2870000.0


In [51]:
x_train, x_test, y_train, y_test = train_test_split(dataframe.drop(columns=OUTPUT_COLUMN), dataframe[OUTPUT_COLUMN])

In [52]:
models = {}

In [53]:
# Линейная регрессия
linear_params = {'n_jobs': range(1, 11)}
linear_model = GridSearchCV(LinearRegression(), linear_params).fit(x_train, y_train).best_estimator_
models['Linear'] = linear_model

In [54]:
# Lasso
params  = {'alpha': range(1, 10)}
lasso = GridSearchCV(Lasso(), params).fit(x_train, y_train).best_estimator_
models['Lasso'] = lasso

In [55]:
# Ridge
ridge_params = {'alpha': range(1, 10), 'solver': ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]}
ridge = GridSearchCV(Ridge(), ridge_params).fit(x_train, y_train).best_estimator_
models['Ridge'] = ridge

In [56]:
# SVR (Support Vector Regression)
params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
svr = GridSearchCV(SVR(), params).fit(x_train, y_train).best_estimator_
models['SVR'] = svr

In [57]:
# Полиномиальная регрессия
model = Pipeline([('poly', PolynomialFeatures()), ('model', GridSearchCV(LinearRegression(), linear_params))])
model.fit(x_train, y_train)
models['Polynomial'] = model

In [58]:
# Байесовская регрессия
params = {'alpha_1': [x/100000 for x in range(1, 5)], 'alpha_2': [x/100000 for x in range(1, 5)],
          'lambda_1': [x/100000 for x in range(1, 5)], 'lambda_2': [x/100000 for x in range(1, 5)]}
model = GridSearchCV(BayesianRidge(), params).fit(x_train, y_train).best_estimator_
models["Bayesian"] = model

In [59]:
# Метод наименьших углов least angle regression (lars)
model = Lars().fit(x_train, y_train)
models['Least angle regression'] = model

In [60]:
# Stochastic Gradient Descent  (SGD)
params = {'loss': ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
          'penalty': ['l1', 'l2', 'elasticnet']}
model = GridSearchCV(SGDRegressor(), params).fit(x_train, y_train).best_estimator_
models['SGDRegression'] = model

In [61]:
# Сплайн
model = Pipeline([('spline', SplineTransformer()), ('model', GridSearchCV(Ridge(), ridge_params))])
model.fit(x_train, y_train)
models['Spline'] = model

In [62]:
# Ядерная регрессия
from sklearn.metrics.pairwise import PAIRWISE_KERNEL_FUNCTIONS 
params = {'kernel': list(PAIRWISE_KERNEL_FUNCTIONS)}
model = GridSearchCV(KernelRidge(), params).fit(x_train, y_train).best_estimator_
models['Kernel'] = model

In [63]:
for name, model in models.items():
    print(f"Модель - {name}; R2 - {r2_score(y_test, model.predict(x_test))}")

print("____")
print("Лучшая модель -", max(models, key = lambda name: r2_score(y_test, models[name].predict(x_test))))


Модель - Linear; R2 - 0.9103146178277863
Модель - Lasso; R2 - 0.910332039967399
Модель - Ridge; R2 - 0.9061273974001411
Модель - SVR; R2 - 0.008565497202941974
Модель - Polynomial; R2 - 0.920215591525346
Модель - Bayesian; R2 - 0.9102705764085809
Модель - Least angle regression; R2 - 0.8722957041169777
Модель - SGDRegression; R2 - 0.008897454721777831
Модель - Spline; R2 - 0.9082687954127947
Модель - Kernel; R2 - 0.9131753470618882
____
Лучшая модель - Polynomial
